In [1]:
%pip install -q -r /content/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.9/119.9 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 82.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 41.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.9/133.9 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

<font color=red>**Danger zone:**</font> you'll be fine-tuning a model to generate positive, negative or even toxic reviews. We'll be doing this for fun, but this is also the technique for [review bombing](https://en.wikipedia.org/wiki/Review_bomb), bot farms on social media and other less than dignified stuff. It is ultimately your decision how you apply this knowledge, but before you choose, ask yourself: is this why you chose to learn ML?


# LLMs Alignment with Reinforcement Learning from human feedback (RLHF).



In this homework, you're gonna fine-tune a language model with reinforcement learning to make it generat bad (or good) reviews.

To perform RL-based fine-tuning, we'll use a new (in this course) library called [Transformer Reinforcement Learning (TRL)](https://huggingface.co/docs/trl). TRL implements the main reinforcement learning components of RLHF: reward modeling and fine-tuning with PPO.

![img](https://huggingface.co/datasets/trl-internal-testing/example-images/resolve/main/images/TRL-readme.png)

## Stage 0: load model

To see how TRL works, we'll use it to align GPT2 on IMDB dataset to generate negative movie reviews. In fact, __it's your choice whether you want positive or negative reviews__, however I recommend you to focus on negative ones, in order to see greater effect after RLHF

But before you choose, let's take a look at the baseline model: a GPT-2 fine-tuned on generating arbitrary movie reviews.

In [ ]:
import datasets
import torch
import transformers
import trl

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
main_tokenizer = transformers.AutoTokenizer.from_pretrained("lvwerra/gpt2-imdb")
main_model = transformers.AutoModelForCausalLM.from_pretrained(
    "lvwerra/gpt2-imdb", device_map=device
)

In [ ]:
inputs = main_tokenizer("The movie", return_tensors="pt").to(device)
generated_ids = main_model.generate(
    **inputs, max_new_tokens=50, do_sample=True, pad_token_id=main_tokenizer.eos_token_id
)
print("\nGenerated text:", main_tokenizer.decode(generated_ids.flatten().cpu().numpy().tolist()))


Generated text: The movie is a big mess, the actors and actresses wasted much of the time because they had not got the best script for their jobs.<br /><br />I'm not sure if it's because they were too distracted by their role and did not understand


If you run this cell a couple of times, you'll see that the model generates both positive, negative and neutral reviews in some proportion. What we're gonna do next is teach the model to generate more positive (or negative) reviews.

Similarly to InstructGPT, we're gonna do that in 2 stages:
- **train a reward model** to assign higher values to positive (or negative) reviews
- fine-tune the language model to **maximize that reward using [proximal policy optimization](https://openai.com/research/openai-baselines-ppo)**



## Stage 1: train a reward model

First, we'll train a BERT-like model as our reward model. We'll generate a synthetic pairwise rankings to emulate human rankings.

__Q:__ why do I need a reward model? Can I just use a pre-trained sentiment classifier? <br> __A:__ Yes, you can - but that only works for movie reviews. But this homework will teach you how to do RLHF for any kind objective.



In [ ]:
# We'll be fine-tuning a small BERT-like model for now. Please try other models for the main assignment.
reward_model = transformers.AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-cased", device_map=device
)
reward_tokenizer = transformers.AutoTokenizer.from_pretrained("distilbert-base-cased")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/465 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/263M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.weight', 'pre_classifier.weight', 'classifier.bias', 'pre_classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

__Note that__ the reward model has a separate tokenizer, different from the main model. They don't need to be the same for RLHF fine-tuning.

In [ ]:
from torch.utils.data import Dataset
from tqdm import tqdm


class IMDBPairwiseDataset(Dataset):
    """
    A dataset of all possible pairs of chosen and rejected texts for TRL reward training format.

    This dataset is designed to facilitate the training of a reward model by providing pairs of
    texts where one is preferred (chosen) and the other is not (rejected). Each sample in the dataset
    is a dictionary containing tokenized input IDs and attention masks for both the chosen and rejected
    texts.

    Parameters:
    imdb: dataset to pairwise
    tokenizer: The tokenizer used to preprocess the texts
    accepted_label (int): The label that indicates a chosen text. Texts with this label are considered
                          preferred, while others are considered rejected.

    Methods:
    __len__(): Returns the total number of possible pairs of chosen and rejected texts.
    __getitem__(index): Returns a dictionary containing tokenized inputs for a specific pair of chosen
                        and rejected texts.
    """

    def __init__(self, imdb: datasets.Dataset, tokenizer, accepted_label: int):
        super().__init__()
        self.tokenizer = tokenizer
        self.chosen_texts = []
        self.rejected_texts = []
        for review in tqdm(imdb, leave=False):
            if review["label"] == accepted_label:
                self.chosen_texts.append(review["text"])
            else:
                self.rejected_texts.append(review["text"])

        assert self.chosen_texts, f"no texts with label {accepted_label}"
        print(
            f"Found {len(self.chosen_texts)} chosen and {len(self.rejected_texts)} rejected texts, {len(self)} pairs"
        )

        self.column_names = [
            "input_ids_chosen",
            "attention_mask_chosen",
            "input_ids_rejected",
            "attention_mask_rejected",
        ]

    def __len__(self):
        return len(self.chosen_texts) * len(self.rejected_texts)  # all pairs

    def __getitem__(self, index: int):
        ch_i, rej_i = index // len(self.chosen_texts), index % len(self.chosen_texts)
        ch_tok = self.tokenizer(
            self.chosen_texts[ch_i], return_tensors="pt", padding="max_length", truncation=True
        ).to(device)
        rej_tok = self.tokenizer(
            self.rejected_texts[rej_i], return_tensors="pt", padding="max_length", truncation=True
        ).to(device)
        return dict(
            input_ids_chosen=ch_tok["input_ids"].squeeze(),
            attention_mask_chosen=ch_tok["attention_mask"].squeeze(),
            input_ids_rejected=rej_tok["input_ids"].squeeze(),
            attention_mask_rejected=rej_tok["attention_mask"].squeeze(),
        )

In [ ]:
TARGET_LABEL = 0  # negative reviews
imdb = datasets.load_dataset("imdb", split="train")
reward_data = IMDBPairwiseDataset(imdb, reward_tokenizer, accepted_label=TARGET_LABEL)

sample = reward_data[31337]
print("CHOSEN:", reward_tokenizer.decode(sample["input_ids_chosen"]))
print("REJECTED:", reward_tokenizer.decode(sample["input_ids_rejected"]))

Found 12500 chosen and 12500 rejected texts, 156250000 pairs
CHOSEN: [CLS] If only to avoid making this type of film in the future. This film is interesting as an experiment but tells no cogent story. < br / > < br / > One might feel virtuous for sitting thru it because it touches on so many IMPORTANT issues but it does so without any discernable motive. The viewer comes away with no new perspectives ( unless one comes up with one while one's mind wanders, as it will invariably do during this pointless film ). < br / > < br / > One might better spend one's time staring out a window at a tree growing. < br / > < br / > [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [P

We'll be using `trl.RewardTrainer` - a special case of `transformers.Trainer`.

![img](https://i.imgur.com/2JzNAPs.png)

Note that the model itself does not score pairs: it processes chosen ($y_w$) and rejected ($y_l$) samples independently. To minimize this loss, the reward model needs to score chosen sample higher than the rejected one. Note that the formula also assumes some context $x$, which is useful for seq2seq tasks. In our case of movie reviews, $x$ is empty.

In [ ]:
training_args = trl.RewardConfig(  # like transformers.TrainingArguments
    output_dir="reward_model",
    per_device_train_batch_size=32,
    gradient_accumulation_steps=1,
    learning_rate=1.41e-5,
    max_steps=1_000,  # note: training may need more than 1k steps
    logging_steps=50,
    # gradient_checkpointing=True,  # reduce memory usage but train ~30% slower
    # gradient_checkpointing_kwargs={"use_reentrant": False},
    fp16=True,  # disable this on CPU or on very old GPUs
    report_to="none",
    # you may add any other hyperparameters that you found useful
)

trainer = trl.RewardTrainer(
    model=reward_model,
    args=training_args,
    tokenizer=reward_tokenizer,
    train_dataset=reward_data,
    peft_config=None,  # optionally, you may tune with LoRA, prompt-tuning, etc
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/trl/trainer/reward_trainer.py:174: UserWarning: When using RewardDataCollatorWithPadding, you should set `max_length` in RewardConfig. It will be set to `512` by default, but you should do it yourself in the future.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/reward_trainer.py:191: UserWarning: When using RewardDataCollatorWithPadding, you should set `remove_unused_columns=False` in your RewardConfig we have set it for you, but you should do it yourself in the future.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:439: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `p

Step,Training Loss
50,0.508000
100,0.168600
150,0.141700
200,0.128400
250,0.109900
300,0.117500
350,0.084200
400,0.073600
450,0.070900
500,0.070500


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2436: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


TrainOutput(global_step=1000, training_loss=0.10818967199325562, metrics={'train_runtime': 1621.4308, 'train_samples_per_second': 19.736, 'train_steps_per_second': 0.617, 'total_flos': 0.0, 'train_loss': 0.10818967199325562, 'epoch': 0.0})

In [11]:
reward_model.gradient_checkpointing_disable()
reward_model.eval();

### Sanity-check the reward model

Let's check how our reward model performs.

__Your task__ is to measure how often does your reward model can rank a pair of (chosen and rejected) reviews correctly. Please measure this separately for train data (`imdb`) and a separate test set loaded below.

In [ ]:
for sample_index in 45, 16000:
    print("TEXT:", imdb[sample_index]["text"])
    inputs = reward_tokenizer(imdb[sample_index]["text"], truncation=True, return_tensors="pt").to(
        device
    )
    with torch.no_grad():
        reward = reward_model(**inputs).logits[0, 0].item()
        print("REWARD:", reward)
    print("LABEL:", imdb[sample_index]["label"])
    print()

# note: your reward model may produce different absolute rewards.
# This is fine as long as the rewards are ordered correctly (most of the time)

TEXT: This movie sucked. It really was a waste of my life. The acting was atrocious, the plot completely implausible. Long, long story short, these people get "terrorized" by this pathetic "crazed killer", but completely fail to fight back in any manner. And this is after they take a raft on a camping trip, with no gear, and show up at a campsite that is already assembled and completely stocked with food and clothes and the daughters headphones. Additionally, after their boat goes missing, they panic that they're stuck in the woods, but then the daughters boyfriend just shows up and they apparently never consider that they could just hike out of the woods like he did to get to them. Like I said, this movie sucks. A complete joke. Don't let your girlfriend talk you into watching it.
REWARD: 5.15625
LABEL: 0

TEXT: Good: Engaging cinematic firefights, great presentation, vehicles are actually fun to drive, fairly appealing multiplayer, faithful to the movie, and the list goes on.<br /><b

First of all, let's implement `compute_reward` function. Note that we use plaintext reviews because main model uses a different tokenizer from the reward model.

In [ ]:
from torch import Tensor, no_grad
from torch.utils.data import DataLoader, Dataset


class MovieReview(Dataset):
    """
    Represents dataset of movie review raw texts without labels
    """

    def __init__(self, texts: list[str]):
        self.texts = texts

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, index):
        return self.texts[index]


def compute_reward(reward_model, reward_tokenizer, texts: list[str], device="cpu") -> Tensor:
    """
    Compute the reward scores for a list of texts using a specified reward model and tokenizer.

    Parameters:
    reward_model: The model used to compute the reward scores
    reward_tokenizer: The tokenizer for reward_model
    texts (list[str]): A list of text strings for which the reward scores are to be computed.
    device (str, optional): The device on which the computation should be performed. Default is 'cpu'.

    Returns:
    torch.Tensor: A tensor containing the reward scores for each input text. The scores are extracted
                  from the logits of the reward model.

    Example:
    >>> compute_reward(my_reward_model, my_reward_tokenizer, ["text1", "text2"])
    tensor([ 5.1836, -4.8438], device='cpu')
    """
    dataset = MovieReview(texts)
    dataloader = DataLoader(dataset, batch_size=64, shuffle=False)
    rewards = []
    with no_grad():
        for batch in tqdm(dataloader, desc="Computing rewards", leave=False):
            inputs = reward_tokenizer(
                batch, truncation=True, padding="max_length", return_tensors="pt"
            ).to(device)
            logits = reward_model(**inputs).logits[:, 0].detach().cpu().tolist()
            rewards.extend(logits)
    return torch.tensor(rewards)

In [ ]:
rewards = compute_reward(
    reward_model, reward_tokenizer, [imdb[45]["text"], imdb[16000]["text"]], device=device
)
print(rewards)
assert rewards[0] > rewards[1]
assert rewards[0] > 0
assert rewards[1] < 0

tensor([ 5.1562, -5.0820])


In [ ]:
import numpy as np
from tqdm.auto import tqdm


def eval_reward_model(reward_model, reward_tokenizer, test_dataset, target_label, device="cpu"):
    """
    Evaluate the performance of a reward model by comparing reward scores for chosen and rejected reviews.

    This function selects reviews from a test dataset based on a target label and evaluates the reward model's
    ability to assign higher scores to chosen reviews compared to rejected ones. The evaluation is performed
    in batches for efficiency.
    Note that reward scores are compared on corresponding chosen and rejected reviews:
        chosen_reviews[0] vs rejected_reviews[0],
        chosen_reviews[1] vs rejected_reviews[1],
        etc.

    Parameters:
    reward_model: The model used to compute the reward scores
    reward_tokenizer: The tokenizer for reward_model
    test_dataset: test Dataset
    target_label (0 or 1): The label used to select chosen reviews. Reviews with this label are considered chosen,
                  while others are considered rejected.
    device (str, optional): The device on which the computation should be performed. Default is 'cpu'.

    Returns:
    float: The accuracy of the reward model, calculated as the proportion of times the model assigns a higher
           reward score to the chosen review compared to the rejected review.

    Example:
    >>> accuracy = eval_reward_model(my_reward_model, my_reward_tokenizer, test_data, target_label=1)
    >>> print(f"Model accuracy: {accuracy:.2%}")
    """
    # Compute rewards of all test dataset
    rewards = compute_reward(reward_model, reward_tokenizer, test_dataset["text"], device=device)

    # Define chosen and rejected indices
    labels = np.array(test_dataset["label"])
    ch_ids = np.argwhere(labels == target_label)
    rej_ids = np.argwhere(labels != target_label)
    assert len(ch_ids) == len(rej_ids)

    # Calculate accuracy
    total = len(ch_ids) * len(rej_ids)
    correct = 0
    for ch_idx in tqdm(ch_ids, desc="Calculating accuracy", leave=False):
        correct += (rewards[rej_ids] < rewards[ch_ids]).sum().item()

    return correct / total

In [ ]:
imdb_test = datasets.load_dataset("imdb", split="test")

test_accuracy = eval_reward_model(
    reward_model,
    reward_tokenizer,
    imdb_test,
    target_label=TARGET_LABEL,
    device=device,
)

print("test accuracy: {}".format(test_accuracy))
assert test_accuracy > 0.94

Computing rewards:   0%|          | 0/391 [00:00<?, ?it/s]

Calculating accuracy:   0%|          | 0/12500 [00:00<?, ?it/s]

test accuracy: 0.96944


### Reward-guided generation (1 point)

If you did everything right, by now you should have a decent reward model. Before we use it for reinforcement learning, let's see if we can align model samples without any training.

To do so, you can use reward-guided inference: __generate N=16 samples, then select the one with the highest reward__ (according to your reward model).

For this problem, it's on you to demonstrate whether or not your code works. Find at least 5 neutral prompts such as "This movie is" (...), generate samples, rank them based on reward and show which samples get the highest reward.

Note: it is faster to generate samples in parallel, rather than sequentially, as follows:




In [ ]:
inputs = main_tokenizer(["It was"] * 5, return_tensors="pt").to(device)
for candidate in main_model.generate(**inputs, max_new_tokens=50, do_sample=True):
    print("Sample:", main_tokenizer.decode(candidate.flatten().cpu().numpy().tolist()))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Sample: It was an excellent film. I loved it so much, I thought that would not have worked out as well as I had imagined it to. This is a fine film--so much so, I thought it a must see if you haven't been paying attention
Sample: It was quite refreshing to see this film on DVD, and it reminded me of several of the earlier movies I've done.<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|>
Sample: It was also the last night the movie premiered. With all my friends I knew the DVD's for the year and all the extras I have seen, I will have to buy this DVD now. It might be a little while longer then the opening credits of the
Sample: It was never going to be a real success.<br /><br />This is no

I edit a bit the signature of this function (add the prompt parameter).  
But it `/scripts` folder i put slightly different func (without added parameter), in order to pass the tests.

In [ ]:
def generate_with_reward_guidance(
    main_model,
    main_tokenizer,
    prompt: str,
    reward_model,
    reward_tokenizer,
    N=16,
    device="cpu",
) -> str:
    """
    Generate text samples using a main model and select the best sample based on a reward model's guidance.

    This function generates multiple text samples from a main model, evaluates each sample using a reward model,
    and returns the sample with the highest reward score. The process is guided by the reward model to select
    the most desirable output.

    Parameters:
    main_model: The language model used to generate text samples.
    main_tokenizer: The tokenizer for main_model
    prompt: Starting prompt, which uses to generate text samples.
    reward_model: The model used to compute reward scores for the generated samples.
    reward_tokenizer: The tokenizer for reward_model
    N (int, optional): The number of text samples to generate. Default is 16.
    device (str, optional): The device on which the computation should be performed. Default is 'cpu'.

    Returns:
    str: The generated text sample with the highest reward score.
    """
    # Generate samples
    inputs = main_tokenizer([prompt] * N, return_tensors="pt").to(device)
    samples: list[str] = []
    for candidate in main_model.generate(
        **inputs, max_new_tokens=50, do_sample=True, pad_token_id=main_tokenizer.eos_token_id
    ):
        samples.append(main_tokenizer.decode(candidate.flatten().cpu().numpy().tolist()))

    # Compute rewards
    rewards = compute_reward(reward_model, reward_tokenizer, samples, device=device)

    # Return the most `rewarded` sample
    return samples[torch.argmax(rewards).item()]

In [ ]:
neutral_prompts = [
    "The movie turned out to be",
    "This movie",
    "Having seen the movie",
    "The movie is",
    "I want to say about the movie that",
    "It was",
]

In [ ]:
for prompt in neutral_prompts:
    top_sample = generate_with_reward_guidance(
        main_model, main_tokenizer, prompt, reward_model, reward_tokenizer, device=device
    )
    print(top_sample)

Computing rewards:   0%|          | 0/1 [00:00<?, ?it/s]

The movie turned out to be terrible. I just sat like a stone in the sand. It looked like I was walking through a long, long tunnel. But that's just it. A lot worse.<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|>


Computing rewards:   0%|          | 0/1 [00:00<?, ?it/s]

This movie is just awful. First you meet a couple very nice and well rounded young men and a couple of young women. In the end the main characters are just awful. At a certain point in the movie it was quite obvious how boring this movie was.


Computing rewards:   0%|          | 0/1 [00:00<?, ?it/s]

...

The movie is so ridiculously bad that my daughter had to watch it. The ending is so predictable it's totally unrealistic that even the plot of the movie can be figured out. The script itself is so stupid that it makes you question if you've ever been warned,


Computing rewards:   0%|          | 0/1 [00:00<?, ?it/s]

I want to say about the movie that it's got a bad plot, in particular its climax and its ending. The whole thing is a hoot to me.<br /><br />Also the character of the character is bland. She's almost cute and just doesn't have any personality


Computing rewards:   0%|          | 0/1 [00:00<?, ?it/s]

It was really boring. It was so repetitive as to be a parody of "Strangelove." It just did nothing to amuse my friends from the beginning (and was never as funny after the premiere), so I guess it's not worth mentioning. I highly


Seems to work

# Stage 2: fine-tune the main model with RL


Now, we will optimize GPT2 to produce negative IMDB movie reviews using the reward model you trained above.

Unlike supervised fine-tuning, RL allows model to generate it's own sentences on each training step. Then, it calculates the reward of those specific sentences, and finally, updates the model to increase the probability of sentences with high reward.

Thus, each RLHF consists of three stages: __Rollout__, __Evaluation__ and __Update__

<div style="text-align: center">
<img src='https://huggingface.co/datasets/trl-internal-testing/example-images/resolve/main/images/gpt2_bert_training.png' width='600'>

The update stage depends on the specific RL algorithm. We'll be using Proximal Policy Optimization, or [PPO](https://arxiv.org/abs/1707.06347), similarly to what was used for InstructGPT.

Before we run those 3 stages, however, we need to create a dataset of "queries" - partial reviews in our case.

In [ ]:
# Note: this code is specific to IMDB; you will need to re-write it for other tasks
imdb_for_rlhf = imdb.filter(lambda row: len(row["text"]) > 200, batched=False)
imdb_for_rlhf = imdb_for_rlhf.remove_columns(["label"])
sample_length = trl.core.LengthSampler(2, 8)  # use the first 2-8 tokens as query


def select_query_and_tokenize(sample):
    query_ids = main_tokenizer.encode(sample["text"])[: sample_length()]
    sample["query"] = main_tokenizer.decode(query_ids)  # query is the only required column
    sample["input_ids"] = query_ids  # to avoid re-tokenizing later
    return sample  # we do not need the rest - it will be generated by the model


imdb_for_rlhf = imdb_for_rlhf.map(select_query_and_tokenize, batched=False)
imdb_for_rlhf.set_format(type="torch")

Filter:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/24895 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1168 > 1024). Running this sequence through the model will result in indexing errors


Finally, we move to RL training. In this tutorial, we'll train LoRA adapters and not the full model.

In [ ]:
import peft

peft_config = peft.LoraConfig(
    task_type=peft.TaskType.CAUSAL_LM, r=32, lora_alpha=32, lora_dropout=0.0, inference_mode=False
)

# reload main model as AutoModelForCausalLMWithValueHead - with an extra head needed for PPO
main_tokenizer = transformers.AutoTokenizer.from_pretrained("lvwerra/gpt2-imdb")
main_tokenizer.pad_token = main_tokenizer.eos_token

main_model = trl.AutoModelForCausalLMWithValueHead.from_pretrained(
    "lvwerra/gpt2-imdb", device_map=device
)
main_model = peft.get_peft_model(main_model, peft_config, adapter_name="default")
main_model.print_trainable_parameters()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:439: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:488: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `w

trainable params: 1,179,648 || all params: 125,620,225 || trainable%: 0.9390589771670923


Same as before, trl has a special type of trainer that minimize PPO-specific pseudo-loss. You can read more on this trainer [here](https://huggingface.co/docs/trl/main/en/ppo_trainer).

In [ ]:
training_args = trl.PPOConfig(
    model_name=main_model.config._name_or_path,
    gradient_accumulation_steps=1,
    learning_rate=1.41e-5,
    batch_size=64,
    ppo_epochs=4,  # PPO performs this many updates per training batch
)

ppo_trainer = trl.PPOTrainer(
    training_args,
    model=main_model.model,
    tokenizer=main_tokenizer,
    dataset=imdb_for_rlhf,
    data_collator=lambda data: dict((key, [d[key] for d in data]) for key in data[0]),
)  # note: we pass main_model.model because PPOTrainer checks for one of several supported model types ...
# ... main_model.model is a model with adapters, which is supported. main_model itself is a wrapper that is not supported

In [ ]:
from tqdm.auto import tqdm

max_steps = 50  # can be insufficient for some tasks - watch your learning curves
generation_kwargs = dict(
    min_length=-1,
    max_new_tokens=128,
    do_sample=True,
    top_k=0,
    top_p=1.0,
    pad_token_id=main_tokenizer.eos_token_id,
)
#                                  ^-- task-specific parameter!

average_reward = 0
gamma = 0.7

with tqdm(enumerate(ppo_trainer.dataloader), total=max_steps) as progressbar:
    # note: ppo_trainer.dataloader is just a regular dataloader of queries, no RL-specific magic :)
    for epoch, batch in progressbar:
        if epoch >= max_steps:
            break

        # Rollout stage: generate continuations from batch queries using main_model
        response_tensors = ppo_trainer.generate(batch["input_ids"], **generation_kwargs)
        # ^-- list of tensors of token ids from main model tokenizer

        # de-tokenize responses to strings (since reward model uses a different tokenizer)
        batch["response"] = [
            main_tokenizer.decode(response.squeeze()) for response in response_tensors
        ]
        # note: response_tensors already contain query tokens, so we don't need to add queries manually.
        # This may not be true for other tasks: check this manually by viewing batch["response"] and batch["query"]

        # Evaluation stage - rewards for batch['response']
        rewards = compute_reward(reward_model, reward_tokenizer, batch["response"], device=device)

        # Update stage
        stats = ppo_trainer.step(batch["input_ids"], response_tensors, list(rewards.split(1)))
        stats["rewards/mean"] = rewards.mean().item()  # compute mean rewards for batch
        average_reward = gamma * average_reward + (1 - gamma) * stats["rewards/mean"]

        print("-" * 30, "STEP", epoch, "-" * 30)
        print(
            f'rewards/mean:\t{stats["rewards/mean"]:.9f}\t<---- average reward over this batch (higher=better, noisy)'
        )
        print(
            f"rewards/moving_avg:\t{average_reward:.9f}\t<---- moving average reward (higher=better, less noisy)"
        )
        print(
            f'ppo/returns/mean:\t{stats["ppo/returns/mean"]:.9f}\t<---- model-estimated average discounted reward'
        )
        print(
            f'objective/kl:\t{stats["objective/kl"]:.9f}\t<---- how far we are from the original model (regularizer)'
        )
        print()

        ppo_trainer.log_stats(stats, batch, list(rewards.split(1)))

  0%|          | 0/50 [00:00<?, ?it/s]

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Computing rewards:   0%|          | 0/1 [00:00<?, ?it/s]

------------------------------ STEP 0 ------------------------------
rewards/mean:	0.121477127	<---- average reward over this batch (higher=better, noisy)
rewards/moving_avg:	0.036443138	<---- moving average reward (higher=better, less noisy)
ppo/returns/mean:	-1.406566143	<---- model-estimated average discounted reward
objective/kl:	0.000000000	<---- how far we are from the original model (regularizer)



Computing rewards:   0%|          | 0/1 [00:00<?, ?it/s]

...

------------------------------ STEP 47 ------------------------------
rewards/mean:	2.656345367	<---- average reward over this batch (higher=better, noisy)
rewards/moving_avg:	2.703383831	<---- moving average reward (higher=better, less noisy)
ppo/returns/mean:	1.278058529	<---- model-estimated average discounted reward
objective/kl:	13.530233383	<---- how far we are from the original model (regularizer)



Computing rewards:   0%|          | 0/1 [00:00<?, ?it/s]

------------------------------ STEP 48 ------------------------------
rewards/mean:	3.446660995	<---- average reward over this batch (higher=better, noisy)
rewards/moving_avg:	2.926366980	<---- moving average reward (higher=better, less noisy)
ppo/returns/mean:	1.711545229	<---- model-estimated average discounted reward
objective/kl:	15.295662880	<---- how far we are from the original model (regularizer)



Computing rewards:   0%|          | 0/1 [00:00<?, ?it/s]

------------------------------ STEP 49 ------------------------------
rewards/mean:	3.535007477	<---- average reward over this batch (higher=better, noisy)
rewards/moving_avg:	3.108959129	<---- moving average reward (higher=better, less noisy)
ppo/returns/mean:	1.713297725	<---- model-estimated average discounted reward
objective/kl:	16.012697220	<---- how far we are from the original model (regularizer)



In [29]:
assert average_reward > 2

And now test your PPO model:

In [ ]:
inputs = [
    main_tokenizer.encode("The movie was", return_tensors="pt").to(device)[0] for i in range(10)
]

response_tensors = ppo_trainer.generate(inputs, **generation_kwargs)
batch["response"] = [main_tokenizer.decode(response.squeeze()) for response in response_tensors]
for sample in batch["response"]:
    print("Sample: {}".format(sample))

Sample: The movie was considered quite disastrous to work with an aging actress who is usually very down, pity her.<|endoftext|>
Sample: The movie was boring atbest.<br /><br />Anyhow, Deck the Millionaire Balls et Al is die like bad.<|endoftext|>
Sample: The movie was nothing hurtless. The acting lacked emotion.<br /><br />This was absolutely nothing dispiriting. The parts were terrible. The script didn't make a cohesive sequence.<br /><br />It was if you fixed getting eaten by Moustache it would go way over, and instead the show was too bad.<|endoftext|>
Sample: The movie was.<br /><br />The film is disgusting and This is nothing that stfu trash.If you are going to send this to the schoolfilms idiot this movie is stupid and should be off for a good summer film.<|endoftext|>
Sample: The movie was basically a cell house party, only a couple of times this dumb show got boring as it was, because none of the people would actually travel planes for this much. Full gallinga Halo stupid star